# Семинар 20
# Методы внутренней точки

## На прошлом семинаре

- Метод проекции градиента
- Метод условного градиента

## Задача выпуклой оптимизации с ограничениями типа равенств

\begin{equation}
\begin{split}
&\min f(x) \\ 
\text{s.t. } & Ax = b,
\end{split}
\end{equation}
где $f$ - выпукла и дважды диференцируема, $A \in \mathbb{R}^{p \times n}$ и $\mathrm{rank} \; A = p < n$

### Двойственная задача
Двойственная функция 
\begin{equation}
\begin{split}
g(\mu) &  = -b^{\top}\mu + \inf_x(f(x) + \mu^{\top}Ax) \\
& = -b^{\top}\mu - \sup_x((-A^{\top}\mu)^{\top}x -f(x)) \\
& = -b^{\top}\mu - f^*(-A^{\top}\mu)
\end{split}
\end{equation}

Двойственная задача
$$
\max_\mu -b^{\top}\mu - f^*(-A^{\top}\mu)
$$

**Подход 1**: найти сопряжённую функцию и решить безусловную задачу оптимизации

**Трудности**
- не всегда легко восстановить решение прямой задачи по решению двойственной
- сопряжённая функция $f^*$ должна быть дважды дифференцируемое для быстрого решения двойственной задачи. Это не всегда так.

### Условия оптимальности

- $Ax^* = b$
- $f'(x^*) + A^{\top}\mu^* = 0$

или
$$
\begin{bmatrix}
f' & A^{\top}\\
A & 0
\end{bmatrix}
\begin{bmatrix}
x^*\\
\mu^*
\end{bmatrix}
 = 
\begin{bmatrix}
0\\
b
\end{bmatrix}
$$

**Подход 2**: решить нелинейную в общем случае систему методом Ньютона.

**Вопрос**: в каком случае система окажется линейной?

## Метод Ньютона для выпуклых задач с ограничениями типа равенств

\begin{equation}
\begin{split}
& \min_v f(x) + f'(x)^{\top}v + \frac{1}{2}v^{\top}f''(x)v\\
\text{s.t. } & A(x + v) = b
\end{split}
\end{equation}

Из условий оптимальности имеем
$$
\begin{bmatrix}
f''(x) & A^{\top}\\
A & 0
\end{bmatrix}
\begin{bmatrix}
v\\
w
\end{bmatrix}
 = 
\begin{bmatrix}
-f'(x)\\
0
\end{bmatrix}
$$

**Шаг метода Ньютона определён только для невырожденной матрицы!**

**Упражнение**. Посчитайте за сколько итераций метод Ньютона сойдётся для квадратичной функции с ограничениями типа равенств.

### Линеаризация условий оптимальности

- $A(x + v) = b \rightarrow Av = 0$
- $f'(x + v) + A^{\top}w \approx f'(x) + f''(x)v + A^{\top}w = 0$

или

- $f''(x)v + A^{\top}w = -f'(x)$

### Псевдокод
**Важно:** начальная точка должна лежать в допустимом множестве!

```python
def NewtonEqualityFeasible(f, gradf, hessf, A, b, stop_crit, line_search, x0, tol):
    x = x0
    n = x.shape[0]
    while True:
        newton_matrix = [[hessf(x), A.T], [A, 0]]
        rhs = [-gradf(x), 0]
        w = solve_lin_sys(newton_matrix, rhs)
        h = w[:n]
        if stop_crit(x, h, gradf(x), **kwargs) < tol:
            break
        alpha = line_search(x, h, f, gradf(x), **kwargs)
        x = x + alpha * h
    return x
```

### Критерий остановки
Получим выражение для значения
$$
f(x) - \inf_v(\hat{f}(x + v) \; | \; A(x+v) = b),
$$
где $\hat{f}$ - квадратичная аппроксимация функции $f$.

Для этого 
$$
\langle h^{\top} \rvert \cdot \quad f''(x)h + A^{\top}w = -f'(x)
$$
с учётом $Ah = 0$ получаем 
$$
h^{\top}f''(x)h = -f'(x)^{\top}h
$$

Тогда 
$$
\inf_v(\hat{f}(x + v) \; | \; A(x+v) = b) = f(x) - \frac{1}{2}h^{\top}f''(x)h
$$

**Вывод:** величина $h^{\top}f''(x)h$ является наиболее адекватным критерием остановки метода Ньютона.

### Теорема сходимости

Сходимость метода аналогична сходимости метода Ньютона для задачи безусловной оптимизации.

**Теорема**
Пусть выполнены следующие условия
- множество уровней $S = \{ x \; | \; x \in D(f), \; f(x) \leq f(x_0), \; Ax = b \}$ замкнуто и $x_0 \in D(f), \; Ax_0 = b$
-  для любых $x \in S$ и $\tilde{x} \in S$ гессиан $f''(x)$ липшицев
- на множестве $S$ $\|f''(x)\|_2 \leq M $ и норма обратной матрицы KKT системы ограничена сверху

Тогда, метод Ньютона сходится к паре $(x^*, \mu^*)$ линейно, а при достижении достаточной близости к решению - квадратично.

## Случай недопустимой начальной точки

- Метод Ньютона требует чтобы начальная точка лежала в допустимом множестве
- Что делать, если поиск такой точки неочевиден: например, если область определения $f$ не сопадает с $\mathbb{R}^n$
- Пусть начальная точка не является допустимой, в этом случае условия KKT можно записать так

$$
\begin{bmatrix}
f''(x) & A^{\top}\\
A & 0
\end{bmatrix}
\begin{bmatrix}
v\\
w
\end{bmatrix}
 = -
\begin{bmatrix}
f'(x)\\
{\color{red}{Ax - b}}
\end{bmatrix}
$$
- Если $x$ допустима, то система совпадает с системой для обычного метода Ньютона

### Прямо-двойственная интерпретация

- Метод *прямо-двойственный*, если на каждой итерации обновляются прямые и двойственные переменные
- Покажем, что это значит. Для этого запишем условия оптимальности в виде
$$
r(x^*, \mu^*) = (r_p(x^*, \mu^*), r_d(x^*, \mu^*)) = 0,
$$
где $r_p(x, \mu) = Ax - b$ и $r_d(x, \mu) = f'(x) + A^{\top}\mu$
- Решим систему методом Ньютона:
$$
r(y + z) \approx r(y) + Dr(y)z = 0
$$

- Прямо-двойственный шаг в методе Ньютона определим как решение линейной системы
$$
Dr(y)z = -r(y)
$$
или более подробно
$$
\begin{bmatrix}
f''(x) & A^{\top}\\
A & 0
\end{bmatrix}
\begin{bmatrix}
z_p\\
z_d
\end{bmatrix}
 = -
\begin{bmatrix}
r_p(x, \mu)\\
r_d(x, \mu)
\end{bmatrix}
= - 
\begin{bmatrix}
f'(x) + A^{\top}\mu\\
Ax - b
\end{bmatrix}
$$

- Заменим $z_d^+ = \mu + z_d$ и получим
$$
\begin{bmatrix}
f''(x) & A^{\top}\\
A & 0
\end{bmatrix}
\begin{bmatrix}
z_p\\
z_d^+
\end{bmatrix}
= - 
\begin{bmatrix}
f'(x)\\
Ax - b
\end{bmatrix}
$$

- Система полностью эквивалентна ранее полученной в обозначениях 

$$
v = z_p \qquad w = z_d^+ = \mu + z_d 
$$

- Метод Ньютона даёт шаг для прямой переменной и обновлённое значение для двойственной

### Способ инициализации

- Удобный способ задания начального приближения: найти точку из области определения $f$ гораздо проще, чем из пересечения области определения и допустимого множества
- Метод Ньютона с недопустимой начальной точкой не может определить согласованность ограничений

### Псевдокод

```python
def NewtonEqualityInfeasible(f, gradf, hessf, A, b, stop_crit, line_search, x0, mu0, tol):
    x = x0
    mu = mu0
    n = x.shape[0]
    while True:
        z_p, z_d = ComputeNewtonStep(hessf(x), A, b)
        if stop_crit(x, z_p, z_d, gradf(x), **kwargs) < tol:
            break
        alpha = line_search(x, z_p, z_d, f, gradf(x), **kwargs)
        x = x + alpha * z_p
        mu = mu + alpha * z_d
    return x
```

### Критерий остановки и линейный поиск

- Изменение $r_p$ после шага $z_p$
$$
A(x + \alpha z_p) - b = [A(x + z_p) = b] = Ax + \alpha(b - Ax) - b = (1 - \alpha)(Ax - b)
$$
- Итоговое изменение после $k$ шагов
$$
r^{(k)} = \prod_{i=0}^{k-1}(1 - \alpha^{(i)})r^{(0)}
$$
- Критерий остановки: $Ax = b$ и $\|r(x, \mu)\|_2 \leq \varepsilon$

- Линейный поиск: $c \in (0, 1/2)$, $\beta = (0, 1)$
```python
def linesearch(r, x, mu, z_p, z_d, c, beta):
    alpha = 1
    while np.linalg.norm(r(x + alpha * z_p, mu + alpha * z_d)) >= (1 - c * alpha) * np.linalg.norm(r(x, mu)):
        alpha *= beta
    return alpha
```

### Теорема сходимости

Результат аналогичен результаты для допустимой начальной точки

**Теорема** Пусть
- множество подуровней $S = \{(x, \mu) \; | \; x \in D(f), \; \| r(x, \mu) \|_2 \leq \| r(x_0, \mu_0)  \|_2 \}$ замкнуто
- на множестве $S$ норма матрицы обратной к ККТ матрице ограничена
- гессиан липшицев на $S$.

Тогда сходимость метода линейна при удалении от решении и квадратичная при достаточном приближении к решению.

## Примеры

## Общая задача выпуклой оптимизации

## Условия оптимальности

## Логарифмический барьер

## Центральный путь

## Физическая интерпретация

## Барьерный метод

### Выбор параметров

### Теорема сходимости

## Задача поиска допустимого начального приближения

### Первая фаза метода

### Вторая фаза метода

### Комбинирование первой фазы и второй фазы

### Общая сложность метода

## Прямо-двойственный метод

### Сравнение с барьерным методом

## Резюме

- Метод Ньютона для выпуклой задачи с оганичениями типа равенств
- Случай недопустимой начальной точки
- Прямой барьерный метод
- Прямо-двойственный метод